In [3]:
# !pip install bert4keras

In [2]:
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer

In [3]:
from keras.layers import Dense
from keras.models import Model
from bert4keras.layers import ConditionalRandomField
from bert4keras.optimizers import Adam

In [4]:
build_transformer_model?

In [40]:
# from bert4keras.models import build_transformer_model
# from bert4keras.tokenizers import Tokenizer
# import numpy as np

# config_path = '/root/kg/bert/chinese_L-12_H-768_A-12/bert_config.json'
# checkpoint_path = '/root/kg/bert/chinese_L-12_H-768_A-12/bert_model.ckpt'
# dict_path = '/root/kg/bert/chinese_L-12_H-768_A-12/vocab.txt'

# tokenizer = Tokenizer(dict_path, do_lower_case=True)  # 建立分词器
# model = build_transformer_model(config_path, checkpoint_path)  # 建立模型，加载权重

# # 编码测试
# token_ids, segment_ids = tokenizer.encode(u'语言模型')

# print('\n ===== predicting =====\n')
# print(model.predict([np.array([token_ids]), np.array([segment_ids])]))

In [42]:
# Category	Privacy
# 0	QQ	527622886 229215757 1827967769 130238288 53344...
# 1	address	西直门 伊朗 红领巾公园 建国门街道 海豐 廣州 智利 深圳 九寨县城 汶川 西西里岛 中国...
# 2	book	《不停的陀螺》 神雕侠侣》 《鹿鼎记》 后汉纪》 《续汉书》 书剑恩仇录 EGM 《洛丽塔》...
# 3	company	CW 索尼 佳士得 必和必拓 必和必拓 暴雪 环球网 育碧 渣打 贝发集团 中诚信国际 美林...
# 4	email	saent@netvigator.com aben@alegendstar.com 3483...
# 5	game	卡米洛特的黑暗时代》 《永恒的任务》 (EverQuest) 《特种部队》 巫妖王之怒》 《...
# 6	government	伊朗外交部 民航昌都站 美国导弹防御局 欧佩克成员国 FTC 美陆军 日军 国军 八路军 伪...
# 7	mobile	010-6362212113910492247 18857150858 00852-6901...
# 8	movie	《加勒比海盗1》 《雪青马》 《花园街五号》 《吐鲁番情歌》 圣诞颂歌》 （achristm...
# 9	name	崔新魁 金庸 英黛安娜 梅赫曼帕拉斯特 孙慧祥先生 孙慧祥 邹力强 胡尔西德.吐尔地 黄刚 ...
# 10	organization	曼联队 桑德兰队 英超联赛 意甲 切沃 民謠乐队 博洛尼亚 那不勒斯 民航西藏区局扶贫办 福...
# 11	position	CEO 专家 古典大师 发言人 北京人 八卦掌高级教练 八卦掌高级教练 开发商 国家一极演员...
# 12	scene	邙山 怪兽公园 魔语怪兽公园 九寨沟 岷江源 川主寺 黄龙松潘古城 牟尼沟 叠溪海子 松坪沟...
# 13	vx	zhuzhiyi91510 1827967769 rachel-213 zhangzhizh...

In [43]:
# def build_transformer_model(
#     config_path=None,  # 模型的配置文件（对应的文件为json格式）
#     checkpoint_path=None,  # 模型的预训练权重（tensorflow的ckpt格式）
#     model='bert',  # 模型的类型（bert、albert、albert_unshared、nezha、electra、gpt2_ml、t5）
#     application='encoder',  # 模型的用途（encoder、lm、unilm）
#     return_keras_model=True,  # 返回Keras模型，还是返回bert4keras的模型类
#     **kwargs  # 其他传递参数
# ):

In [27]:
model_roberta=build_transformer_model(
    vocab_size=30000,
    hidden_size=206,#SEQ长度
    num_hidden_layers=3,#transformers个数
    num_attention_heads=3,#一个transformers中的MultiHeadSelfAttent个数
    intermediate_size=64,#Transformer-1-FeedForward 层即fc层的大小
    hidden_act="gelu",
    model="roberta"
)

In [28]:
model_roberta.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 206)    6180000     Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 206)    412         Input-Segment[0][0]              
___________________________________________________________________________________________

In [30]:
output_roberta = model_roberta.get_layer('Transformer-2-FeedForward-Norm').output

In [32]:
output_fin = Dense(6)(output_roberta)
output_fin

<tf.Tensor 'dense_218/add:0' shape=(None, None, 6) dtype=float32>

# 基本形式

In [37]:
bert_layers = 3
num_labels = 12
lr_multiplier=1000
model = build_transformer_model(
    vocab_size=30000,
    hidden_size=128,
    num_hidden_layers=3,
    num_attention_heads=2,
    intermediate_size=128,
    hidden_act="gelu"
    )

output_layer = 'Transformer-%s-FeedForward-Norm' % (bert_layers - 1)
output = model.get_layer(output_layer).output
output = Dense(num_labels)(output)
CRF = ConditionalRandomField(lr_multiplier=lr_multiplier)
output = CRF(output)

model = Model(model.input, output)
model.summary()

model.compile(
    loss=CRF.sparse_loss,
    optimizer=Adam(0.01),
    metrics=[CRF.sparse_accuracy]
)

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 128)    3840000     Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 128)    256         Input-Segment[0][0]              
___________________________________________________________________________________________

In [39]:
# model.input?

# 改进

begin: 0000000010000000000

end:   0000000000100000000

cat:   0000000033300000000

In [45]:
model2 = build_transformer_model(
    vocab_size=30000,
    hidden_size=128,
    num_hidden_layers=3,
    num_attention_heads=2,
    intermediate_size=128,
    hidden_act="gelu"
    )

output_layer = 'Transformer-%s-FeedForward-Norm' % (bert_layers - 1)
output = model2.get_layer(output_layer).output
#两个不同学习率的CRF
output1 = Dense(10)(output)
CRF1 = ConditionalRandomField(lr_multiplier=100)
output1 = CRF1(output1)

output2 = Dense(9)(output)
CRF2 = ConditionalRandomField(lr_multiplier=1)
output2 = CRF2(output2)
model2 = Model(model2.input, [output1,output2])

model2.compile(
    loss=[CRF1.sparse_loss,CRF2.sparse_loss],
    optimizer=Adam(0.01),
    metrics=[CRF1.sparse_accuracy,CRF2.sparse_accuracy]
)

In [46]:
model2.summary()

Model: "model_22"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 128)    3840000     Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 128)    256         Input-Segment[0][0]              
___________________________________________________________________________________________

In [42]:
CRF2.sparse_loss?

# 改进三

In [7]:
model3 = build_transformer_model(
    vocab_size=30000,
    hidden_size=128,
    num_hidden_layers=3,
    num_attention_heads=2,
    intermediate_size=128,
    hidden_act="gelu"
    )

output_layer = 'Transformer-%s-FeedForward-Norm' % (bert_layers - 1)
output = model3.get_layer(output_layer).output
#CRF 和Dense softmax层融合
output1 = Dense(10)(output)
CRF1 = ConditionalRandomField(lr_multiplier=1)
output1 = CRF1(output1)

output2 = Dense(units=9,activation='softmax')(output)
model3 = Model(model3.input, [output1,output2])

In [9]:
model3.summary()
model3.compile(
    loss=[CRF1.sparse_loss,'sparse_categorical_crossentropy'],
    optimizer=Adam(0.01)
)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 128)    3840000     Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 128)    256         Input-Segment[0][0]              
____________________________________________________________________________________________

# 改进四

In [48]:
from keras.layers import Lambda
import tensorflow as tf

In [49]:
def concat(dense_list):
    dense = tf.concat(dense_list, axis=-1)
    return dense

In [50]:
model4 = build_transformer_model(
    vocab_size=30000,
    hidden_size=128,
    num_hidden_layers=3,
    num_attention_heads=2,
    intermediate_size=128,
    hidden_act="gelu"
    )

output_layer = 'Transformer-%s-FeedForward-Norm' % (bert_layers - 1)
output = model4.get_layer(output_layer).output

#两分支【bertoutput->CRF1_output】【 output,CRF1_output ->output2_crf】

# 辅助的label
output1 = Dense(10)(output)
CRF1 = ConditionalRandomField(lr_multiplier=1)
output1_crf = CRF1(output1)

# 主要的label
concat_output1_output = Lambda(concat)([output,output1])
CRF2 = ConditionalRandomField(lr_multiplier=1)
output2_crf = CRF2(concat_output1_output)

In [51]:
model4 = Model(model4.input, [output1_crf,output2_crf])
model4.compile(
    loss=[CRF1.sparse_loss,CRF2.sparse_loss],
    optimizer=Adam(0.01),
    metrics=[CRF1.sparse_accuracy,CRF2.sparse_accuracy]
)

In [52]:
model4.summary()

Model: "model_25"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 128)    3840000     Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 128)    256         Input-Segment[0][0]              
___________________________________________________________________________________________

# 调参数